In [34]:
import pandas as pd 
import numpy as np 
import sys

file_name="\测试数据.xlsx"
file_path=r"F:\OneDrive\华莱士\电商"
file=file_path+file_name
df = pd.read_excel(file,
                   usecols = "D,M,R:AD",
                   sheet_name= '订单明细')
df.head(10)

,交易类型,配送方式,商家应收款,商品总价,餐盒费,打包袋,商家活动支出,美团活动补贴,平台服务费,公益捐款,用户支付配送费,用户线上支付金额,用户线下支付金额,费率,保底
0,外卖订单,美团专送,27.20,37.0,1.0,0.0,-6.0,-,-4.80,0.0,5.0,37.0,0.0,15.0%,4.5
1,外卖订单,到店取餐,10.45,20.0,0.0,0.0,-9.0,-,-0.55,0.0,0.0,11.0,0.0,5.0%,0.5
2,津贴联盟,NaN,-1.21,0.0,0.0,0.0,0.0,-,0.00,0.0,0.0,0.0,0.0,NaN,-
3,外卖订单,美团专送,18.90,24.4,1.0,0.0,-2.0,-,-4.50,0.0,3.0,26.4,0.0,15.0%,4.5
4,订单退款,美团专送,-5.50,-22.0,0.0,0.0,12.0,-1.00,4.50,0.0,3.0,-12.0,0.0,15.0%,4.5
5,订单部分退款,美团专送,-2.51,0.0,0.0,0.0,0.0,-,0.00,0.0,0.0,0.0,0.0,NaN,-
6,外卖订单,代理商配送,22.44,41.0,0.0,0.0,-12.0,6.50,-6.56,0.0,4.0,26.5,0.0,16.0%,-
7,外卖订单,代理商配送,36.32,48.0,2.0,0.0,-6.0,1.50,-7.68,0.0,4.0,46.5,0.0,16.0%,-
8,订单部分退款,代理商配送,-7.38,0.0,0.0,0.0,0.0,-,0.00,0.0,0.0,0.0,0.0,NaN,-
9,订单退款,代理商配送,-12.16,-24.0,-1.0,0.0,9.0,-6.50,3.84,0.0,4.0,-13.5,0.0,16.0%,-


In [35]:
# 设定保底和费率
设定保底=4.5
设定费率=15

# 测算保底和费率
测算保底=0 # 这个没保底的
测算费率=20

print(
"""
配送方式	费率	保底
美团快送	15.0%	4.5
美团专送	15.0%	4.5
到店取餐	5.0%	0.5
津贴联盟		-
代理商配送	16.0%	-按原价
商家自配	16.0%	-
只测算美团的
""")

df.dtypes


配送方式	费率	保底
美团快送	15.0%	4.5
美团专送	15.0%	4.5
到店取餐	5.0%	0.5
津贴联盟		-
代理商配送	16.0%	-按原价
商家自配	16.0%	-
只测算美团的



交易类型         object
配送方式         object
商家应收款       float64
商品总价        float64
餐盒费         float64
打包袋         float64
商家活动支出      float64
美团活动补贴       object
平台服务费       float64
公益捐款        float64
用户支付配送费     float64
用户线上支付金额    float64
用户线下支付金额    float64
费率           object
保底           object
dtype: object

In [36]:
# 版本0.21.0引入了infer_objects()方法，用于将具有对象数据类型的DataFrame的列转换为更具体的类型。

df['美团活动补贴']=df['美团活动补贴'].replace('-',0).astype(float)
df['用户支付配送费']=df['用户支付配送费'].replace('-',0).astype(float)
df['用户线上支付金额']=df['用户线上支付金额'].replace('-',0).astype(float)
try:
    df['费率']=df['费率'].str.strip("%").astype(float) # Python如何将百分号的字符转成数字
except AttributeError as err:
    print(err)

df['保底']=df['保底'].replace('-',0).astype(float) # Python强行转换格式

df美团待测算=df

df美团待测算.dtypes

交易类型         object
配送方式         object
商家应收款       float64
商品总价        float64
餐盒费         float64
打包袋         float64
商家活动支出      float64
美团活动补贴      float64
平台服务费       float64
公益捐款        float64
用户支付配送费     float64
用户线上支付金额    float64
用户线下支付金额    float64
费率          float64
保底          float64
dtype: object

In [37]:
print("当前最大费率（代理商费率16，美团专送15）：",df['费率'].max(),"当前最大保底：",df['保底'].max())
print("""
费率	计数
""",df美团待测算['费率'].value_counts())

当前最大费率（代理商费率16，美团专送15）： 16.0 当前最大保底： 4.5

费率	计数
 15.0    12
16.0     8
6.0      3
10.0     2
5.0      1
Name: 费率, dtype: int64


In [38]:
print("描述性统计，数据收集目标：保底订单计数，总订单计数，全部客单价、保底客单价、抽点客单价（筛选美团外卖）")

df美团待测算.loc[df美团待测算['平台服务费']== -df美团待测算['保底'],'是否保底']=True # 算保底
df美团待测算['是否保底']=df美团待测算['是否保底'].fillna(False)
df美团待测算.head(10)

描述性统计，数据收集目标：保底订单计数，总订单计数，全部客单价、保底客单价、抽点客单价（筛选美团外卖）


,交易类型,配送方式,商家应收款,商品总价,餐盒费,打包袋,商家活动支出,美团活动补贴,平台服务费,公益捐款,用户支付配送费,用户线上支付金额,用户线下支付金额,费率,保底,是否保底
0,外卖订单,美团专送,27.20,37.0,1.0,0.0,-6.0,0.0,-4.80,0.0,5.0,37.0,0.0,15.0,4.5,False
1,外卖订单,到店取餐,10.45,20.0,0.0,0.0,-9.0,0.0,-0.55,0.0,0.0,11.0,0.0,5.0,0.5,False
2,津贴联盟,NaN,-1.21,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,NaN,0.0,True
3,外卖订单,美团专送,18.90,24.4,1.0,0.0,-2.0,0.0,-4.50,0.0,3.0,26.4,0.0,15.0,4.5,True
4,订单退款,美团专送,-5.50,-22.0,0.0,0.0,12.0,-1.0,4.50,0.0,3.0,-12.0,0.0,15.0,4.5,False
5,订单部分退款,美团专送,-2.51,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,NaN,0.0,True
6,外卖订单,代理商配送,22.44,41.0,0.0,0.0,-12.0,6.5,-6.56,0.0,4.0,26.5,0.0,16.0,0.0,False
7,外卖订单,代理商配送,36.32,48.0,2.0,0.0,-6.0,1.5,-7.68,0.0,4.0,46.5,0.0,16.0,0.0,False
8,订单部分退款,代理商配送,-7.38,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,NaN,0.0,True
9,订单退款,代理商配送,-12.16,-24.0,-1.0,0.0,9.0,-6.5,3.84,0.0,4.0,-13.5,0.0,16.0,0.0,False


In [39]:
df描述性统计1=df美团待测算.groupby(['交易类型','配送方式','是否保底'])['平台服务费'].count().reset_index()
df描述性统计1['统计']='计数'
df描述性统计1['文件名']=file_name

In [40]:
df描述性统计2=df美团待测算.groupby(['交易类型','配送方式','是否保底'])['平台服务费'].mean().reset_index()
df描述性统计2['统计']='平均值'
df描述性统计2['文件名']=file_name

In [41]:
df描述性统计3=df美团待测算.groupby(['交易类型','配送方式','是否保底'])['平台服务费'].sum().reset_index()
df描述性统计3['统计']='求和'
df描述性统计3['文件名']=file_name

In [42]:
pd.concat([df描述性统计1,df描述性统计2,df描述性统计3],axis=0).to_csv(file_path + file_name +  '输出' + '.csv')

In [28]:
# 服务费计算方法
# df美团专送.loc[df美团专送['校验']>=1] 这样会出来一堆代理商配送，这种代理商配送的，是原价*费率，我们一定要谈掉，太坑爹了
# df美团待测算['无保底服务费']=(df美团待测算['用户线上支付金额'] - df美团待测算['用户支付配送费'] + df美团待测算['美团活动补贴'])*设定费率/100
